<a href="https://colab.research.google.com/github/hhindley/phd/blob/main/colab_notebooks/fmfn_bayesopt_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install bayesian-optimization
# !pip install scipy==1.7.3
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
from scipy.integrate import odeint
from scipy.integrate import ode
from scipy.integrate import solve_ivp
from pandas import DataFrame

In [ ]:
def func(K1_tag, K2_tag, K1_rep, K2_rep, w_ab):
  params = [L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, max, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, d, krep, kdam, ktag, kdeg, kin, atp]
  sol = solve_ivp(model, [t0, tf], init, args=(params,), method='Radau', t_eval=np.arange(60))
  return np.sum(np.square(sol.y[1][-1]-data.loc[10][1]))

In [ ]:
def func1(K1_tag, K2_tag, K1_rep, K2_rep, w_ab):
  # params = [L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, max, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, d, krep, kdam, ktag, kdeg, kin, atp]
  
  # kdam = 0.2 - oxidative damage so increase in rate of damage to RNA - same effect as if adding antibiotics 
  sol_gor = solve_ivp(model, [t0, tf], init, args=([L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, max, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, d, krep, 0.2, ktag, kdeg, kin, atp],), method='Radau')#, t_eval=np.arange(60))
  gor = np.sum(np.square(sol_gor.y[1][-1]-data.loc[10][1]))

  # kdeg = 0.0001; d = 0.001 - ribonuclease so less degradation of RNA - specifically mRNA but can also be rRNA f 
  sol_mazf = solve_ivp(model, [t0, tf], init, args=([L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, max, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, 0.001, krep, kdam, ktag, 0.0001, kin, atp],), method='Radau')#, t_eval=np.arange(60))
  mazf = np.sum(np.square(sol_mazf.y[1][-1]-data.loc[10][2]))

  # max = 1 - deacylase in tRNA editing so faulty translation - slower production of proteins
  sol_ybak = solve_ivp(model, [t0, tf], init, args=([L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, 1, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, d, krep, kdam, ktag, kdeg, kin, atp],), method='Radau')#, t_eval=np.arange(60))
  ybak = np.sum(np.square(sol_ybak.y[1][-1]-data.loc[10][3]))

  # kdam = 0.2 - important in cell envelope stress so more oxidative stress maybe so increase in rate of damage to RNA
  sol_yobf = solve_ivp(model, [t0, tf], init, args=([L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, max, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, d, krep, 0.2, ktag, kdeg, kin, atp],), method='Radau')#, t_eval=np.arange(60))
  yobf = np.sum(np.square(sol_yobf.y[1][-1]-data.loc[10][4]))

  return (0.4*gor+ 0.2*mazf+0.5*ybak+ 0.8*yobf)

In [ ]:
optimizer = BayesianOptimization(func1, param_range, random_state=27, verbose=2) # verbose = 1 prints only when a maximum is observed (pink)
optimizer.maximize(init_points=2, n_iter=15, acq='ucb', kappa=5)

In [ ]:
K1_tag = 1; K2_tag = 20; K1_rep = 1; K2_rep = 20; atp = 20;
params = [L, c, kr, Vmax_init, Km_init, w_ab, w_r, θ, max, thr, K1_tag, K2_tag, K1_rep, K2_rep, gr_c, d, krep, kdam, ktag, kdeg, kin, atp]
sol = solve_ivp(model, [t0, tf], init, args=(params,), method='Radau', t_eval=np.arange(60))


In [ ]:
np.sum(np.square(sol.y[1]-syn_data.y[1]))